<a href="https://colab.research.google.com/github/Sudhanshu-Mittal-2024/Deep_Learning_Projects/blob/main/distracted_driver_multi_action_project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q -O driver_detection.zip https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=0

In [2]:
!unzip -q /content/driver_detection.zip

In [3]:
import os
import shutil
os.makedirs('/content/driver_detection')
shutil.move('/content/imgs.zip','/content/driver_detection')
shutil.move('/content/driver_imgs_list.csv','/content/driver_detection')

'/content/driver_detection/driver_imgs_list.csv'

In [4]:
!unzip -q /content/driver_detection/imgs.zip -d /content/driver_detection/

In [5]:
!rm -rf '/content/driver_detection/imgs.zip'
!rm -rf '/content/driver_detection.zip'

In [6]:
os.listdir('/content/driver_detection')

['train', 'driver_imgs_list.csv', 'test']

In [7]:
train_floder_path='/content/driver_detection/train/'
test_folder_path='/content/driver_detection/test/'

In [8]:
train_sub_dir=os.listdir(train_floder_path)
filenames = []
for cl in train_sub_dir:
  filenames = filenames + [train_floder_path+cl+'/'+file for file in os.listdir(train_floder_path+cl+'/')]

len(filenames)

22424

In [9]:
import pandas as pd
import numpy as np
training_data=pd.read_csv('/content/driver_detection/driver_imgs_list.csv')
training_data.shape

(22424, 3)

In [10]:
training_data.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [11]:
for _,row in training_data.iterrows():
  training_data['img_path']=train_floder_path+training_data['classname']+'/'+training_data['img']

In [12]:
training_data.head()

,subject,classname,img,img_path
0,p002,c0,img_44733.jpg,/content/driver_detection/train/c0/img_44733.jpg
1,p002,c0,img_72999.jpg,/content/driver_detection/train/c0/img_72999.jpg
2,p002,c0,img_25094.jpg,/content/driver_detection/train/c0/img_25094.jpg
3,p002,c0,img_69092.jpg,/content/driver_detection/train/c0/img_69092.jpg
4,p002,c0,img_92629.jpg,/content/driver_detection/train/c0/img_92629.jpg


In [13]:
import matplotlib.pyplot as plt

In [14]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import preprocess_input

In [15]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
from tensorflow.keras.models import Model

In [17]:
x = base_model.output
global_pool = GlobalAveragePooling2D(name='global_avg_pooling')(x)
dense1 = Dense(128, activation='relu', name='dense1')(global_pool)
dropout = Dropout(0.2, name='dropout')(dense1)

output = Dense(10, activation='softmax', name='output')(dropout)
model = Model(inputs=base_model.input, outputs=output)

In [18]:
for layer in base_model.layers:
  layer.trainable = False

In [19]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
train_floder_path,test_folder_path

('/content/driver_detection/train/', '/content/driver_detection/test/')

In [23]:
#!rm -rf '/content/driver_detection/test/.ipynb_checkpoints/'

In [24]:
#!rm -rf '/content/driver_detection/test/c0'
#!rm -rf '/content/driver_detection/test/c1'
#!rm -rf '/content/driver_detection/test/c2'
#!rm -rf '/content/driver_detection/test/c3'
#!rm -rf '/content/driver_detection/test/c4'
#!rm -rf '/content/driver_detection/test/c5'
#!rm -rf '/content/driver_detection/test/c6'
#!rm -rf '/content/driver_detection/test/c7'
#!rm -rf '/content/driver_detection/test/c8'
#!rm -rf '/content/driver_detection/test/c9'

In [25]:
move_file_list=[]
move_file_list=os.listdir(test_folder_path)
len(move_file_list)

for cl in train_sub_dir:
  os.makedirs(test_folder_path+cl)

for _,row in training_data.iterrows():
  if row['img'] in move_file_list:
    shutil.move(test_folder_path+row['img'],test_folder_path+row['classname']+'/')

In [26]:
len(move_file_list)

79726

In [27]:
for image in move_file_list:
  shutil.move(test_folder_path+image, test_folder_path+'c0/')

In [28]:
len(os.listdir(test_folder_path))

10

In [29]:
#for _,row in training_data.iterrows():
#  if row['img'] == 'img_1.jpg':
#    print(row['classname']+row['img'])
#  else :
#    print("Not Found")

In [30]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [31]:
train_generator = train_datagen.flow_from_directory(train_floder_path,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical',
                                                    shuffle=False
                                                    )
test_generator = test_datagen.flow_from_directory(test_folder_path,
                                                    target_size=(224,224),
                                                    batch_size=16,
                                                    class_mode='categorical')

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 10 classes.


In [33]:
steps_per_epoch = train_generator.n // train_generator.batch_size // 10
validation_steps = test_generator.n // test_generator.batch_size // 100

steps_per_epoch, validation_steps

(140, 49)

In [34]:
model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=3,
                    validation_data=test_generator,
                    validation_steps=validation_steps
          )

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


140/140 ━━━━━━━━━━━━━━━━━━━━ 541s 4s/step - accuracy: 0.1296 - loss: 2.6650 - val_accuracy: 0.2449 - val_loss: 2.0404
Epoch 2/3
140/140 ━━━━━━━━━━━━━━━━━━━━ 529s 4s/step - accuracy: 0.1145 - loss: 2.3206 - val_accuracy: 0.0000e+00 - val_loss: 2.1687
Epoch 3/3
140/140 ━━━━━━━━━━━━━━━━━━━━ 524s 4s/step - accuracy: 0.0716 - loss: 2.3030 - val_accuracy: 0.9796 - val_loss: 2.2116


In [35]:
from tensorflow.keras.preprocessing import image

In [36]:
def process_image_for_validation(my_image):
  img=image.load_img(my_image,target_size=(224,224))
  x=image.img_to_array(img)
  x=preprocess_input(x)
  x=np.expand_dims(x,axis=0)
  return x

In [37]:
def predict_image(my_image):
  prediction = model.predict(process_image_for_validation(my_image))
#  predicted_class = np.argmax(prediction)
#  class_labels = {v: k for k, v in train_generator.class_indices.items()}
#  print(f"{my_image} ------Prediction Class from Model: {class_labels[predicted_class]} Having Probability: {prediction[0, predicted_class]:.2f}")
  return prediction

In [44]:
train_generator.class_indices

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}

In [43]:
final_pred=[]
for my_image in move_file_list:
  final_pred.append(predict_image(test_folder_path+'c0/'+my_image))

final_pred = np.concatenate(final_pred) # Convert list of arrays to single array

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

KeyboardInterrupt: 

In [49]:
submissions=pd.DataFrame({'c0-c9':final_pred})

submissions.to_csv('/content/submission.csv',index=False)